In [1]:
from pymatgen.io.vasp import Poscar
import pymatgen as pmg
from itertools import count
import numpy as np
from collections import Counter

In [2]:
class Species():
    #Initiates count when initialise() isn't called
    _core_shell_count = count(1)
    
    def __init__(self, name, mass, charge, core_shell=False, shell_mass=None):
        self.name = name
        self.mass = mass
        self.core_shell = core_shell
        if core_shell:
            self.core_shell_index = next(self._core_shell_count)
            if not shell_mass:
                shell_mass = self.mass * 0.1
            self.core_mass = self.mass - shell_mass
            self.shell_mass = shell_mass
            self.types = [AtomType(mass=self.core_mass, core_shell='core', charge=charge['core']),
                          AtomType(mass=self.shell_mass, core_shell='shell', charge=charge['shell'])]
        else:
            self.types = [AtomType(mass=self.mass, core_shell='core', charge=charge)]
            
    @classmethod
    def initialise(cls):
        Species._core_shell_count = count(1)

class AtomType():
    #Initiates count when initialise() isn't called
    _atom_type_count = count(1)
    
    def __init__(self, mass, charge, core_shell=None):  
        self.mass = mass
        self.charge = charge
        self.index = next(self._atom_type_count)
        self.core_shell = core_shell

    @classmethod
    def initialise(cls):
        AtomType._atom_type_count = count(1)
        
class Atoms():
    def __init__(self, index, moleculeID, type_num, charge, mass, sites, element, atom_type):
        self.index = index
        self.moleculeID = moleculeID
        self.type_num = type_num
        self.charge = charge
        self.mass = mass
        self.sites = sites
        self.element = element
        self.label = '{}_{}'.format(element.name,atom_type)
        self.core = True
        if atom_type is 'shell':
            self.core = False
        self.lammps_atoms = '{:>4} {:>4} {:>3} {:7.3f} {:10.6f} {:10.6f} {:10.6f}\n'.format(
            index, moleculeID, type_num, charge, *sites)
    
class System():
    def __init__(self, poscar, species, atoms, core_shell, lammps_file=False):
        self.poscar = poscar
        self.species = species
        self.atoms = atoms
        self.core_shell = core_shell
        self.lammps_file = lammps_file
        
        self.natom_types = len(set([atom.type_num for atom in self.atoms]))
        self.natoms = len(atoms)
        self.nbonds = len([1.0 for atom in atoms if not atom.core])
        self.nbond_types = len([ 1.0 for s in self.species if self.species[s].core_shell])
        self.lattice = poscar.structure.lattice.abc
        
        
    def to_file(self, lammps_file=None):
        masses = [atom.mass for atom in self.atoms]
        atom_masses = sorted(set(masses), key=masses.index)
        labels = sorted(set([atom.label for atom in self.atoms]))
        if self.lammps_file:
            with open(self.lammps_file, 'w') as out:
                out.write('{}\n\n'.format(self.poscar.comment))
                out.write('{}   atoms\n{}   bonds\n\n'.format(self.natoms, self.nbonds))
                out.write('{}   atom types\n{}   bond types\n\n'.format(self.natom_types, self.nbond_types))
                out.write('0.0 {:2.6f} xlo xhi\n0.0 {:2.6f} ylo yhi\n0.0 {:2.6f} zlo zhi\n\n'.format(*self.lattice))
                out.write('Masses\n\n')
                [out.write('{} {:9.5f} # {}\n'.format(i+1, mass, labels[i])) for i, mass in enumerate(atom_masses)] 
                out.write('\nAtoms\n\n')
                [out.write(atom.lammps_atoms) for atom in self.atoms]
                if self.core_shell:
                    out.write('\nBonds\n\n')
                    i = 0
                    for atom in self.atoms:
                        spec = species[atom.element.name]
                        if not atom.core and spec.core_shell:
                            i+=1
                            out.write('{:>4}  {:>4}  {:>4}  {:>4}\n'.format(
                                i,spec.core_shell_index, atom.index-1, atom.index))

In [3]:
def get_data(poscar, core_shell, charges):
    AtomType.initialise() #Required when using a notebook, not if command line called.
    elements = Counter(poscar.structure.species)
    species = { e.name: Species(name=e.name,
                                mass=e.atomic_mass,
                                charge=charges[e.name],
                                core_shell=core_shell[e.name] ) 
                for e, num_atoms in elements.items() }
    
    atoms = []
    i = 0 # counter for atom index
    for m, s in enumerate(poscar.structure.sites):
        spec = species[s.specie.name]
        for t in spec.types:
            i += 1
            atoms.append(Atoms(index=i, moleculeID=m+1, type_num=t.index,
                               charge=t.charge, mass=t.mass, sites=s.coords,
                               element=spec, atom_type=t.core_shell))
    
    system = System(poscar=poscar, species=species, atoms=atoms, core_shell=core_shell,
                    lammps_file=lammps_file)
    
    return(elements, species, system)

In [4]:
poscar = Poscar.from_file('POSCAR1')
core_shell = { 'Li': False , 'Ni': False, 'O': True}
charges = {'Li': +1.0,
           'Ni': +3.0,
           'O': {'core':  +0.960,
                 'shell': -2.960}}
lammps_file='lammps_out.txt'

In [5]:
elements, species, system = get_data(poscar, core_shell, charges)

In [6]:
system.to_file(lammps_file)